# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [308]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [309]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [310]:
# frequency table for prestige and whether or not someone was admitted
freq_table = pd.crosstab(index=df['admit'], columns = df['prestige'])

freq_table.index = ['not admitted', 'admitted']
freq_table.columns = ['1', '2', '3', '4']
freq_table.T

,not admitted,admitted
1,28,33
2,95,53
3,93,28
4,55,12


## Part 2. Return of dummy variables

#### 2.1 Create class of dummy variables for prestige 

In [311]:
dummy_var = pd.get_dummies(df['prestige'])
dummy_var.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
df_new = df.join(dummy_var)
df_new.head()

,admit,gre,gpa,prestige,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4.0,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need the number of levels (i.e. # of prestige categories) minus one, meaning that 3 variables would be necessary.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [312]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_var.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         0.0         1.0         0.0
1      1  660.0  3.67         0.0         0.0         1.0         0.0
2      1  800.0  4.00         1.0         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         0.0         1.0


In [313]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

freq_table2 = pd.crosstab(index=df_new['admit'], columns = df_new['prestige_1'])

freq_table2.index = ['not admitted', 'admitted']
freq_table2.columns = ['did not attend #1 ranked college', 'did attend  #1 ranked college']
freq_table2.T


,not admitted,admitted
did not attend #1 ranked college,243,93
did attend #1 ranked college,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [314]:
Odds_attend_1 = (33./28)
Odds_attend_1

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [315]:
Odds_noattend_1 = (93./243)
Odds_noattend_1

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [316]:
OR_1 = Odds_attend_1/Odds_noattend_1
OR_1

3.079493087557604

#### 3.4 Write this finding in a sentence: 

Answer: Given that the odds ratio is greater than 1, this shows us that there is indeed an association between prior attendance to a #1 ranked college and higher odds of admission to graduate school.

#### 3.5 Print the cross tab for prestige_4

In [317]:
freq_table3 = pd.crosstab(index=df_new['admit'], columns = df_new['prestige_4'])

freq_table3.index = ['not admitted', 'admitted']
freq_table3.columns = ['did not attend #4 ranked college', 'did attend  #4 ranked college']
freq_table3.T

,not admitted,admitted
did not attend #4 ranked college,216,114
did attend #4 ranked college,55,12


#### 3.6 Calculate the OR 

In [318]:
Odds_attend_4 = (12./55)
Odds_noattend_4 = (114./216)
OR_2 = Odds_attend_4/Odds_noattend_4
OR_2

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: As can be seen above, the odds ratio is below 1. This implies that there is little to no association between prior attendance at a #4 ranked college and higher odds of admission to graduate school.

## Part 4. Analysis

In [319]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_var.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61         0.0         1.0         0.0
1      1  660.0  3.67         0.0         1.0         0.0
2      1  800.0  4.00         0.0         0.0         0.0
3      1  640.0  3.19         0.0         0.0         1.0
4      0  520.0  2.93         0.0         0.0         1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [320]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [361]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [322]:
logit = sm.Logit(data['admit'], data[train_cols])
results = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [323]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Wed, 17 Aug 2016   Pseudo R-squ.:                 0.08166
Time:                        20:45:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [324]:
print np.exp(results.params)

gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
intercept     0.020716
dtype: float64


In [325]:
params = results.params
conf = results.conf_int()
conf['OR'] = params
conf.columns = ['2.5', '97.5', 'OR']
print np.exp(conf)

                 2.5      97.5        OR
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525
intercept   0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds ratio of Prestige_2, given that it is lower than 1, shows that given prior attendance at a #2 ranked school is associated with lower odds of admission to a graduate school. 

#### 4.6 Interpret the OR of GPA

Answer: The odds ratio of GPA is greater than 1, meaning that there is an association between GPA and higher odds of admission to graduate school.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [334]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [373]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [374]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_var = pd.get_dummies(df['prestige'])
dummy_var.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_var.ix[:, 'prestige_2':])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1.0,1.0,0.0,1.0,0.0
1,220.0,2.260000,2.0,1.0,0.0,1.0,0.0
2,220.0,2.260000,3.0,1.0,0.0,0.0,0.0
3,220.0,2.260000,4.0,1.0,0.0,0.0,1.0
4,220.0,2.453333,1.0,1.0,0.0,0.0,1.0


In [389]:
dummy_var.prestige_1.value_counts()

0.0    336
1.0     61
Name: prestige_1, dtype: int64

#### 5.2 Make predictions on the enumerated dataset

In [385]:
combos['admit_pred'] = results.predict(combos[train_cols])
combos.tail()
combos[(combos.gre == 800) & (combos.gpa == 4.00) 
       & (combos.prestige_2 == 0.0)
       & (combos.prestige_3 == 0.0)
       & (combos.prestige_4 == 0.0)]

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_pred


#### 5.3 Interpret findings for the last 4 observations

Answer: ?